# Preparación de los datos

Añadiendo indicadores como:


*   MACD (Moving Average Convergence Divergence)
*   RSI (Relative Strength Index)
*    CMF (Chaikin Money Flow)

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('/content/googl_daily_prices.csv')
dataset.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0


In [ ]:
dataset.columns = [col.split('. ')[1] if '. ' in col else col for col in dataset.columns]

In [4]:
dataset.head()

,date,open,high,low,close,volume
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0


In [6]:
df = dataset
df.head()

,date,open,high,low,close,volume
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0


📊 MACD (Moving Average Convergence Divergence)

In [8]:
ema_12 = df['close'].ewm(span=12, adjust=False).mean()
ema_26 = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema_12 - ema_26
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['Hist'] = df['MACD'] - df['Signal']

In [9]:
df.head()

,date,open,high,low,close,volume,MACD,Signal,Hist
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0,0.000000,0.000000,0.000000
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0,-0.390883,-0.078177,-0.312707
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0,-0.726978,-0.207937,-0.519041
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0,-1.095292,-0.385408,-0.709884
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0,-1.341860,-0.576698,-0.765161


📉 RSI (Relative Strength Index)

In [10]:
n = 14
delta = df['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(n).mean()
avg_loss = loss.rolling(n).mean()

rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

In [11]:
df.head()

,date,open,high,low,close,volume,MACD,Signal,Hist,RSI
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0,0.000000,0.000000,0.000000,NaN
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0,-0.390883,-0.078177,-0.312707,NaN
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0,-0.726978,-0.207937,-0.519041,NaN
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0,-1.095292,-0.385408,-0.709884,NaN
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0,-1.341860,-0.576698,-0.765161,NaN


💰 CMF (Chaikin Money Flow)

In [12]:
mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'])
mfv = mfm * df['volume']
df['CMF'] = mfv.rolling(20).sum() / df['volume'].rolling(20).sum()


In [13]:
df.head()

,date,open,high,low,close,volume,MACD,Signal,Hist,RSI,CMF
0,2025-08-08,197.22,202.6100,197.170,201.42,39161826.0,0.000000,0.000000,0.000000,NaN,NaN
1,2025-08-07,197.06,197.5400,194.330,196.52,26321811.0,-0.390883,-0.078177,-0.312707,NaN,NaN
2,2025-08-06,194.50,196.6300,193.670,196.09,21562908.0,-0.726978,-0.207937,-0.519041,NaN,NaN
3,2025-08-05,194.71,197.8599,193.885,194.67,31602340.0,-1.095292,-0.385408,-0.709884,NaN,NaN
4,2025-08-04,190.29,195.2700,190.120,195.04,31547350.0,-1.341860,-0.576698,-0.765161,NaN,NaN


In [14]:
df.to_csv('googl_daily_prices_indicadores.csv', index=False)